In [ ]:
import pandas as pd
import re

## Dados de Venda

In [ ]:
dados_vendas = pd.read_table('dados_vendas.txt')
dados_vendas

In [ ]:
dados_vendas.info()

In [ ]:
dados_vendas.isna().sum()

## Dados do Produto

In [ ]:
dados_produtos = pd.read_table('dados_produtos.txt')
dados_produtos

In [ ]:
dados_produtos.info()

In [ ]:
dados_produtos.isna().sum()

## Etapas da limpeza

1. consertar os id's em ambas as bases

2. renomear as colunas na base **VENDAS**

3. consertar nomes dos produtos nas bases

4. substituir valores NA/NaN e consertar nomes da base **PRODUTOS**

5. fazer um join das bases

6. transformar wide ---> long para facilitar cálculos com o tempo

## Etapa 1

In [ ]:
# dados_vendas
dados_vendas['ID_FERTILIZANTE'] = [s[-2:] for s in dados_vendas['ID_FERTILIZANTE']]
dados_vendas

In [ ]:
# dados_produtos
dados_produtos['ID_FERTILIZANTE'] = [s[-2:] for s in dados_produtos['ID_FERTILIZANTE']]
dados_produtos

In [ ]:
dados_vendas.sort_values(by='ID_FERTILIZANTE')

In [ ]:
# removendo nomes e deixando apenas os id's
dados_produtos.drop(columns=['NOME'], inplace=True)
dados_produtos

## Etapas 2 e 3

In [ ]:
dados_vendas.rename(columns={'FATURAMENTO_JAN_23':1,
                            'FATURAMENTO_FEV_23':2,
                            'FATURAMENTO_MAR_23':3,
                            'FATURAMENTO_ABR_23':4,
                            'FATURAMENTO_MAI_23':5,
                            'FATURAMENTO_JUN_23':6,
                            'FATURAMENTO_JUL_23':7,
                            'FATURAMENTO_AGO_23':8,
                            'FATURAMENTO_SET_23':9,
                            'FATURAMENTO_OUT_23':10,
                            'FATURAMENTO_NOV_23':11,
                            'FATURAMENTO_DEZ_23':12}, inplace=True)
dados_vendas

In [ ]:
dados_vendas['NOME'] = [s.strip() for s in dados_vendas['NOME']]
dados_vendas

## Etapa 4

In [ ]:
dados_vendas.isna().sum()

In [ ]:
dados_vendas.fillna(0, inplace=True)
dados_vendas

In [ ]:
dados_produtos.replace({'SintÃ©tico':'Sintático', 'OrgÃ¢nico':'Orgnânico', 'MÃ©dia':'Média', 'MÃ©dio':'Médio', 'AspersÃ£o':'Aspersão', 'RÃ¡pido':'Rápido'}, inplace=True)
dados_produtos

In [ ]:
# base vendas
dados_vendas['PRECO_MEDIO_GALAO_2022'] = [float(re.sub(',', '.', s)) for s in dados_vendas['PRECO_MEDIO_GALAO_2022']]
dados_vendas['PRECO_MEDIO_GALAO_2023'] = [float(re.sub(',', '.', s)) for s in dados_vendas['PRECO_MEDIO_GALAO_2023']]
dados_vendas

## Etapa 5

In [ ]:
dados_cruzados = dados_vendas.merge(dados_produtos, on='ID_FERTILIZANTE')
dados_cruzados

In [ ]:
dados_cruzados.info()

## Etapa 6

In [ ]:
id_vars = list(dados_cruzados.iloc[:,:4].columns) + list(dados_cruzados.iloc[:,16:].columns)
id_vars

In [ ]:
value_vars = list(dados_cruzados.iloc[:,4:16])
value_vars

In [ ]:
dados_melt = dados_cruzados.melt(id_vars=id_vars, value_vars=value_vars, value_name='Faturamento', var_name='Mês')
dados_melt

In [ ]:
dados_melt.groupby(['Mês'])['Faturamento'].agg(['sum', 'mean'])

In [ ]:
dados_melt.to_csv('VendaProduto.csv', index=False)